In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from datetime import datetime, timedelta
from unittest.mock import MagicMock, patch

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
                                  # asset: (decision_px, exec_px, qty)
positions = OrderedDict()

positions[datetime(2011, 1, 1)] = {'Asset1': (100, 101, 2)}
positions[datetime(2011, 1, 2)] = {
                                    'Asset1': (101, 102, 1), 
                                    'Asset2': (201, 202, 3),
                                    'Asset3': (301, 302, 4)
                                  }

positions[datetime(2011, 1, 3)] = {'Asset1': (102, 103, 1), 'Asset2': (202, 203, 0)}


In [ ]:
for dt, pos_list in positions.items():
    print(f'{dt}: {pos_list}')

# Calculation of transactions

In [ ]:
%%timeit 
transactions = OrderedDict()
prev_pos = None
for dt, pos_list in positions.items():
    if prev_pos is None:
        transactions[dt] = pos_list
    else:
        for p_asset, p_values in pos_list.items():
            prev_values = prev_pos.get(p_asset, None)
            t_dict = transactions.setdefault(dt, {})
            
            if prev_values:
                # Calculating transactions for existing position
                t_qty = p_values[2] - prev_values[2]
                if t_qty != 0:
                    t_dict[p_asset] = (p_values[0], p_values[1], t_qty)
                pass
            else:
                
                t_dict[p_asset] = p_values
    
    prev_pos = pos_list
        

In [ ]:
[print(f'{dt}: {pos_list}') for dt,pos_list in transactions.items()];

In [ ]:
class PosRec(object):
    __slots__ = ['decision_px', 'exec_px', 'qty']
    def __init__(self, decision_px, exec_px, qty):
        self.decision_px = decision_px
        self.exec_px = exec_px
        self.qty = qty

In [ ]:
def calc_transactions(current_pos, prev_pos):
    result = {}
    if prev_pos is None:
        intersected_assets = set(current_pos)
    else:
        intersected_assets = set(current_pos) | set(prev_pos)
        
    for asset in intersected_assets:        
        prev_values = prev_pos.get(asset, None) if prev_pos is not None else None
        curr_values = current_pos.get(asset, None)
        
        if prev_values is None:
            result[asset] = (curr_values[0], curr_values[1], curr_values[2], 0.0, 0.0)
        elif curr_values is None:
            # TODO: call get price from DM here!
            result[asset] = (0, 0, -prev_values[2], 0.0, 0)
        else:
            # Calculating transactions for existing position
            # TODO: replace by asset static method
            pnl_decision = (curr_values[0] - prev_values[0]) * prev_values[2]
            pnl_execution = (curr_values[1] - prev_values[1]) * prev_values[2]
            
            result[asset] = (curr_values[0], curr_values[1], curr_values[2] - prev_values[2], pnl_decision, pnl_execution)
        
    return result
        

In [ ]:
%lprun -f calc_transactions calc_transactions(positions[datetime(2011, 1, 3)], positions[datetime(2011, 1, 2)])

In [ ]:

transactions = OrderedDict()
prev_pos = None
for dt, pos_list in positions.items():
    transactions[dt] = calc_transactions(pos_list, prev_pos)    
    prev_pos = pos_list

In [ ]:
[print(f'{dt}: {pos_list}') for dt,pos_list in positions.items()];

In [ ]:
[print(f'{dt}: {pos_list}') for dt,pos_list in transactions.items()];

In [ ]:
def transactions_stats(trans_dict):
    pnl_change_decision = 0.0
    pnl_change_execution = 0.0
    nfutures_executed = 0.0
    noptions_executed = 0.0
    
    for asset, trans in trans_dict.items():
        pnl_change_decision += trans[3]
        pnl_change_execution += trans[4]
        
        if asset == 'Asset1':
            nfutures_executed += abs(trans[2])
        else:
            noptions_executed += abs(trans[2])
            
    return {
        'pnl_change_decision': pnl_change_decision,
        'pnl_change_execution': pnl_change_execution,
        'nfutures_executed': nfutures_executed,
        'noptions_executed': noptions_executed,
    }
        
def merge_actual_position(actual_position, trans_dict):
    for asset, trans in trans_dict.items():
        t_dpx, t_epx, t_qty, _, _ = trans
        act_pos = actual_position.get(asset, None)
        
        if not act_pos:
            if t_qty != 0:
                actual_position[asset] = (t_dpx, t_epx, t_qty, t_dpx*t_qty, t_epx*t_qty)
        else:
            p_wavg_decision, p_wavg_exec, p_qty, p_wsum_decision, p_wsum_exec = act_pos
            p_wsum_decision += t_dpx*t_qty
            p_wsum_exec += t_epx*t_qty
            p_qty += t_qty
            if p_qty == 0:
                del actual_position[asset]
            else:
                p_wavg_decision = p_wsum_decision / p_qty 
                p_wavg_exec = p_wsum_exec / p_qty

                actual_position[asset] = (p_wavg_decision, p_wavg_exec, p_qty, p_wsum_decision, p_wsum_exec)
            

In [ ]:
#%%timeit 
pnl_result = []
actual_position_result = {}
prev_pos = None

for dt, pos_list in positions.items():
    transactions = calc_transactions(pos_list, prev_pos)    
    
    stats = transactions_stats(transactions)
    
    merge_actual_position(actual_position_result, transactions)
    print(f"Actual position for {dt}: ")
    print(actual_position_result)
    
    res = {'dt': dt}
    res.update(stats)
    
    pnl_result.append(res)    
    prev_pos = pos_list

In [ ]:
pd.DataFrame(pnl_result)

In [ ]:
%timeit (20, 10 ,10 )

In [ ]:
from array import array
%timeit array('f', (20, 10 ,10 ,10 ,10))

In [ ]:
arr = array('f', (20, 10 ,10 ,10 ,10))
%timeit arr[0]


In [ ]:
pos_dict1 = OrderedDict()
for i in range(3000):
    pos_dict1[datetime(2011, 1, 1) + timedelta(days=i)] = (1, 2, 3)

In [ ]:
%timeit pos_dict1[datetime(2012, 1, 1)]

In [ ]:
%timeit pos_dict1.keys()

In [ ]:
from bisect import bisect_right

In [ ]:
bisect_right(pos_dict1.keys(), datetime(2012, 1, 1))

In [ ]:
d = {'d': 'ds'}

In [ ]:
if d:
    print('ok')